# Convolutional Neural Networks

## Convolution

$$
[H]_{i,j,c_{out}} = \sum_{c_{in}} \sum_{a} \sum_{b} [V]_{a,b,c_{in},c_{out}} \cdot [X]_{i+a,j+b,c_{in}}

In [59]:
import numpy as np

def convolve(image: np.ndarray, kernel: np.ndarray, /, stride: int = 1, padding: int = 0, channels: int = 1):
    if type(stride) is int:
        stride: tuple[int, int] = (stride, stride)
        
    if type(padding) is int:
        padding: tuple[int, int] = (padding, padding)
    
    ic, ih, iw = image.shape
    kh, kw = kernel.shape
    ph, pw = padding
    sh, sw = stride
    
    out_shape = ( channels, (ih + 2*ph - kh) // sh + 1, (iw + 2*pw - kw) // sw + 1) 
    output = np.zeros(out_shape)
    
    aux_shape = ( ic, (ih + 2*ph - kh) // sh + 1, (iw + 2*pw - kw) // sw + 1)
    aux = np.zeros(aux_shape)
    print(aux_shape)
    
    image = np.pad(image, ((0, 0), padding, padding), 'constant')
    
    for c in range(ic):
        for i in range(0, ih + 2*ph - kh + 1, sh):
            for j in range(0, iw + 2*pw - kw + 1, sw):
                aux[c, i//sh, j//sw] = np.sum(image[c, i:i+kh, j:j+kw] * kernel)
    
    # for i in range(0, ih + 2*ph - kh, sh):
    #     for j in range(0, iw + 2*pw - kw, sw):
    #         output[i//sh, j//sw] = np.sum(image[i:i+kh, j:j+kw] * kernel)
            
    return aux

In [63]:
input = np.array([
    [[0, 1, 2],
    [3, 4, 5],
    [6, 7, 8]],
    
    [[1, 2, 3],
    [4, 5, 6],  
    [7, 8, 9]]
])

kernel = np.array([
    [0, 1],
    [2, 3]
])

convolve(input, kernel, stride=1, padding=0)

(2, 2, 2)


array([[[19., 25.],
        [37., 43.]],

       [[25., 31.],
        [43., 49.]]])

## Pooling

In [54]:
def pool(image: np.ndarray, fn, /, size: int = 2, stride: int = 1):
    if type(stride) is int:
        stride: tuple[int, int] = (stride, stride)
        
    if type(size) is int:
        size: tuple[int, int] = (size, size)
    
    sth, stw = stride
    szh, szw = size
    
    ic, ih, iw = image.shape
    out_shape = (ic, (ih - szh) // sth + 1, (iw - szw) // stw + 1)
    output = np.zeros(out_shape)
    
    for c in range(ic):
        for i in range(0, ih - szh + 1, sth):
            for j in range(0, iw - szw + 1, stw):
                output[c, i//sth, j//stw] = fn(image[c, i:i+szh, j:j+szw])
                
    return output

def max_pool(image: np.ndarray, /, size: int = 2, stride: int = 1):
    return pool(image, np.max, size=size, stride=stride)

def avg_pool(image: np.ndarray, /, size: int = 2, stride: int = 1):
    return pool(image, np.mean, size=size, stride=stride)

In [55]:
image = np.array([
    [[0, 1, 2],
    [3, 4, 5],
    [6, 7, 8]]
])

max_pool(image, size=2, stride=1)

array([[[4., 5.],
        [7., 8.]]])